# Current Work

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os
import json

In [153]:
# notebook settings
pd.set_option('display.max_columns', None)

In [154]:
path = "./unsynced-data/crops/"
crop_files = os.listdir(path)
crop_files = [f for f in crop_files if f.endswith('.csv')]
grid = gpd.read_file("./synced-data/empty-grid.geojson")


In [155]:
def reduce_simulations(df, string, grid_df):
    working_df = df[["Exp.", "Yield", "Biomass", "Duration", "lat", "long"]].groupby("Exp.").mean().reset_index()
    # clean up and turn df into geodataframe
    working_df.columns = map(str.lower, working_df.columns)
    working_df = working_df.rename(columns={"yield": "yield_" + string, "biomass": "biomass_" + string, "duration": "duration_" + string})
    working_df = gpd.GeoDataFrame(working_df, crs="EPSG:4326", geometry=gpd.points_from_xy(working_df['long'], working_df['lat']))
    working_df = working_df.drop(columns=["lat", "long", "exp."])
    # add working_df to grid_df where geometries intersect on geometry
    grid_df = gpd.sjoin(grid_df, working_df, how="left", op="intersects")
    grid_df = grid_df.drop(columns='index_right')
    grid_df.columns = map(str.lower, grid_df.columns)
    return grid_df

In [156]:
def reduce_models(df):
    # create a list of column names
    column_names = df.columns.tolist()
    column_names.remove("geometry")
    models = ["_ipsl", "_mpi", "_gfdl", "_mri"]
    
    for m in models:
        # remove  m from the column names
        column_names = [c.replace(m, "") for c in column_names]
    # remove duplicates in column_names list by converting to set and back to list
    column_names = list(set(column_names))
    # average all columns that match a column name
    for c in column_names:
        df[c] = df.filter(regex=c).mean(axis=1)
    # drop all columns that include a string from models
    df = df.drop(columns=[c for c in df.columns if any(m in c for m in models)])
    df = df.round(3)
    return df


In [157]:
# It feels like there should be a better way to remove/not write NAN's to a geojson file to save space
def geojson_no_nans(df, path):
    df.to_file(path, driver='GeoJSON')
    # Load the GeoJSON data from a file
    with open(path, 'r') as geojson_file:
        geojson_data = json.load(geojson_file)

    def remove_null_properties(feature):
        non_null_properties = {key: value for key, value in feature['properties'].items() if value is not None}
        feature['properties'] = non_null_properties

    # Loop through features and remove null properties
    for feature in geojson_data['features']:
        remove_null_properties(feature)

    # Save the modified GeoJSON to a new file
    with open(path, 'w') as output_file:
        json.dump(geojson_data, output_file, indent=2)

In [8]:
# a dictionary of column quanitiles is useful for styling the map in the frontend
def generate_quantiles(df):
    col_list = df.columns.tolist()
    col_list.remove("geometry")
    q_list = ["min", 0.01, 0.02, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.98, 0.99, "max"]
    working_dict = {}
    for col in col_list:
        quantile_dict = {}
        for quantile in q_list:
            if quantile == "min":
                quantile_value = df[col].min()
                quantile_dict[quantile] = quantile_value

            elif quantile == "max":
                quantile_value = df[col].max()
                quantile_dict[quantile] = quantile_value
            else:
                quantile_value = df[col].quantile(quantile)
                quantile_dict[f"quantile_{int(quantile * 100)}"] = quantile_value  
        working_dict[col] = quantile_dict
    return working_dict

In [159]:
for c in crop_files:
    file_df = pd.read_csv(path + c)
    column_string = c.split("-")[0]
    grid = reduce_simulations(file_df, column_string, grid)
grid = grid.round(3)


/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/inter

In [160]:
geojson_no_nans(grid, "./synced-data/crop-yields-all-models.geojson")

In [161]:
grid_reduced = reduce_models(grid)

In [162]:
geojson_no_nans(grid_reduced, "./synced-data/crop-yields-mean-models.geojson")

In [9]:
quantiles = generate_quantiles(grid_reduced)
quantiles
# I copy this output into the dataQuantiles.js file

{'biomass_taro_future_ssp126': {'min': 11.158,
  'quantile_1': 99.48701999999999,
  'quantile_2': 168.298,
  'quantile_10': 573.8752000000001,
  'quantile_20': 1201.7744000000002,
  'quantile_30': 3082.6451999999986,
  'quantile_40': 5687.4764000000005,
  'quantile_50': 9304.9,
  'quantile_60': 13215.632999999994,
  'quantile_70': 15490.542,
  'quantile_80': 16714.993000000002,
  'quantile_90': 19253.022000000004,
  'quantile_98': 21606.900999999998,
  'quantile_99': 21921.5605,
  'max': 22517.892},
 'duration_taro_future_ssp126': {'min': 40.4,
  'quantile_1': 43.45096,
  'quantile_2': 45.07204,
  'quantile_10': 63.3888,
  'quantile_20': 83.6984,
  'quantile_30': 93.89699999999998,
  'quantile_40': 102.4652,
  'quantile_50': 116.333,
  'quantile_60': 132.11119999999997,
  'quantile_70': 142.0666,
  'quantile_80': 149.76680000000002,
  'quantile_90': 162.71640000000002,
  'quantile_98': 199.77208,
  'quantile_99': 208.30006000000003,
  'max': 211.0},
 'duration_tef_historical_ssp370': {